[Kinetica](https://www.kinetica.com/) is a database with integrated support for vector similarity search.

`KineticaStore` is a `ByteStore` implementation backed by a Kinetica table. It stores key-value pairs where keys are strings and values are bytes, making it suitable for caching, document storage, and other use cases that require persistent key-value access.

This notebook shows how to use the `KineticaStore` key-value store.

This needs an instance of Kinetica which can easily be set up using the instructions given here - [installation instruction](https://www.kinetica.com/developer-edition/).

In [1]:
pip install -qU langchain-kinetica

/Users/chadjuliano/git/langchain-kinetica/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


You must set the database connection in the following environment variables. If you are using a virtual environment you can set them in the `.env` file of the project:

* `KINETICA_URL`: Database connection URL (e.g. `http://localhost:9191`)
* `KINETICA_USER`: Database user
* `KINETICA_PASSWD`: Secure password.

In [2]:
from gpudb import GPUdb

kdbc = GPUdb.get_connection(enable_ssl_cert_verification=True)

2026-02-26 16:54:27.719 INFO     [GPUdb] Connected to Kinetica! (host=https://demo72.kinetica.com/_gpudb api=7.2.3.6 server=7.2.3.8)


## Instantiating the store

Create a `KineticaStore` by providing a database connection, a schema name, and a collection name. Setting `delete_existing_collection=True` drops any pre-existing table with the same name so that the notebook starts from a clean state.

In [3]:
from langchain_kinetica.storage import KineticaStore

SCHEMA_NAME = "test"
COLLECTION_NAME = "langchain_kv_store_example"

store = KineticaStore(
    kdbc=kdbc,
    schema_name=SCHEMA_NAME,
    collection_name=COLLECTION_NAME,
    delete_existing_collection=True,
)
store

## Writing values

Use `mset` to write one or more key-value pairs. Values must be `bytes`.

In [4]:
store.mset(
    [
        ("user:1", b"Alice"),
        ("user:2", b"Bob"),
        ("user:3", b"Charlie"),
        ("config:theme", b"dark"),
        ("config:language", b"en"),
    ]
)
print("Values written successfully.")

Values written successfully.


## Reading values

Use `mget` to retrieve values for a list of keys. Missing keys return `None`.

In [5]:
keys = ["user:1", "user:2", "user:3", "user:99"]
values = store.mget(keys)

for key, value in zip(keys, values, strict=False):
    print(f"{key!r}: {value}")

'user:1': b'Alice'
'user:2': b'Bob'
'user:3': b'Charlie'
'user:99': None


## Listing keys

Use `yield_keys` to iterate over all keys in the store. Pass an optional `prefix` argument to filter the results.

In [6]:
print("All keys:")
for key in store.yield_keys():
    print(f"  {key}")

print()
print("Keys with prefix 'user:':")
for key in store.yield_keys(prefix="user:"):
    print(f"  {key}")

print()
print("Keys with prefix 'config:':")
for key in store.yield_keys(prefix="config:"):
    print(f"  {key}")

All keys:
  user:1
  user:2
  user:3
  config:theme
  config:language

Keys with prefix 'user:':
  user:1
  user:2
  user:3

Keys with prefix 'config:':
  config:theme
  config:language


## Updating values

`mset` performs an upsert: calling it again with an existing key replaces the stored value.

In [7]:
# Update user:1 and add a new key
store.mset([("user:1", b"Alicia"), ("user:4", b"Diana")])

updated = store.mget(["user:1", "user:4"])
print(f"user:1 -> {updated[0]}")
print(f"user:4 -> {updated[1]}")

user:1 -> b'Alicia'
user:4 -> b'Diana'


## Deleting keys

Use `mdelete` to remove one or more keys from the store.

In [8]:
store.mdelete(["user:3", "user:4"])

# Verify deletion — deleted keys return None
values = store.mget(["user:1", "user:2", "user:3", "user:4"])
for key, value in zip(["user:1", "user:2", "user:3", "user:4"], values, strict=False):
    print(f"{key!r}: {value}")

'user:1': b'Alicia'
'user:2': b'Bob'
'user:3': None
'user:4': None


## Using KineticaStore as a cache backend

`KineticaStore` implements `ByteStore`, which makes it compatible with `CacheBackedEmbeddings`. This caches computed embeddings in Kinetica so that repeated embedding calls for the same text are served from the database rather than re-computed.

In [9]:
import getpass
import os

from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

underlying_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

cache_store = KineticaStore(
    kdbc=kdbc,
    schema_name=SCHEMA_NAME,
    collection_name="langchain_embedding_cache",
    delete_existing_collection=True,
)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=underlying_embeddings,
    document_embedding_cache=cache_store,
    namespace=underlying_embeddings.model,
)

texts = [
    "Kinetica is a high-performance analytical database.",
    "LangChain makes building LLM applications easy.",
    "Vector similarity search enables semantic retrieval.",
]

# First call — embeddings are computed and cached in Kinetica
embeddings_1 = cached_embeddings.embed_documents(texts)
print(f"Computed {len(embeddings_1)} embeddings (first call — cache miss).")

# Second call — embeddings are served from the Kinetica cache
embeddings_2 = cached_embeddings.embed_documents(texts)
print(f"Retrieved {len(embeddings_2)} embeddings (second call — cache hit).")

# Inspect cached keys
print()
print("Cached embedding keys (first 3):")
for key in list(cache_store.yield_keys())[:3]:
    print(f"  {key}")

/Users/chadjuliano/git/langchain-kinetica/.venv/lib/python3.13/site-packages/langchain_classic/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


Computed 3 embeddings (first call — cache miss).
Retrieved 3 embeddings (second call — cache hit).

Cached embedding keys (first 3):
  text-embedding-3-larged5d650fb-e5ea-5641-b47d-01fb06f7cb96
  text-embedding-3-large71f58d85-5df8-5964-a9f9-1cc28a3f78e4
  text-embedding-3-largea3189ce9-0af5-5b23-b208-420663cd8e22
